In [6]:
!pip install transformers sentencepiece evaluate -q
!pip install sacrebleu
import evaluate
from transformers import (
    MarianTokenizer, MarianMTModel,
    MBart50TokenizerFast, MBartForConditionalGeneration,
    M2M100Tokenizer, M2M100ForConditionalGeneration
)





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.6 MB/s eta 0:00:00


In [2]:
#Models

# MarianMT (English -> French)
marian_model_name = "Helsinki-NLP/opus-mt-en-fr"
marian_tokenizer = MarianTokenizer.from_pretrained(marian_model_name)
marian_model = MarianMTModel.from_pretrained(marian_model_name)

# mBART-50 (Multilingual)
mbart_model_name = "facebook/mbart-large-50-many-to-many-mmt"
mbart_tokenizer = MBart50TokenizerFast.from_pretrained(mbart_model_name)
mbart_model = MBartForConditionalGeneration.from_pretrained(mbart_model_name)

# M2M100 (Universal Multilingual)
m2m_model_name = "facebook/m2m100_418M"
m2m_tokenizer = M2M100Tokenizer.from_pretrained(m2m_model_name)
m2m_model = M2M100ForConditionalGeneration.from_pretrained(m2m_model_name)




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [3]:
#  Translation Functions

def translate_marian(text):
    inputs = marian_tokenizer(text, return_tensors="pt", padding=True)
    translated = marian_model.generate(**inputs)
    return marian_tokenizer.decode(translated[0], skip_special_tokens=True)

def translate_mbart(text):
    mbart_tokenizer.src_lang = "en_XX"
    encoded = mbart_tokenizer(text, return_tensors="pt")
    generated_tokens = mbart_model.generate(**encoded, forced_bos_token_id=mbart_tokenizer.lang_code_to_id["fr_XX"])
    return mbart_tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

def translate_m2m(text):
    m2m_tokenizer.src_lang = "en"
    encoded = m2m_tokenizer(text, return_tensors="pt")
    generated_tokens = m2m_model.generate(**encoded, forced_bos_token_id=m2m_tokenizer.get_lang_id("fr"))
    return m2m_tokenizer.decode(generated_tokens[0], skip_special_tokens=True)


In [10]:
  #Input Sentences

sentences = [
    "The Eiffel Tower is located in Paris.",
    "I am learning machine translation.",
    "This is a good day to study NLP."
]


# Translations

print("\n TRANSLATION COMPARISON \n")
results = []

for text in sentences:
    marian_out = translate_marian(text)
    mbart_out = translate_mbart(text)
    m2m_out = translate_m2m(text)
    results.append((text, marian_out, mbart_out, m2m_out))

    print(f" EN: {text}")
    print(f" MarianMT: {marian_out}")
    print(f" mBART-50: {mbart_out}")
    print(f" M2M100  : {m2m_out}")
    print("-" * 90)






 TRANSLATION COMPARISON 

 EN: The Eiffel Tower is located in Paris.
 MarianMT: La Tour Eiffel est située à Paris.
 mBART-50: La Tour Eiffel est située à Paris.
 M2M100  : La tour Eiffel est située à Paris.
------------------------------------------------------------------------------------------
 EN: I am learning machine translation.
 MarianMT: J'apprends la traduction automatique.
 mBART-50: Je suis en train d'apprendre la traduction par machine.
 M2M100  : J’apprends la traduction machine.
------------------------------------------------------------------------------------------
 EN: This is a good day to study NLP.
 MarianMT: C'est une bonne journée pour étudier le NLP.
 mBART-50: C'est un bon jour pour étudier le NL.
 M2M100  : C'est un bon moment pour étudier le NLP.
------------------------------------------------------------------------------------------


In [11]:
# BLEU Score Evaluation

print("\n BLEU SCORE COMPARISON ")
bleu = evaluate.load("sacrebleu")

# Sample reference translations
references = [
    "La tour Eiffel est située à Paris.",
    "J’apprends la traduction automatique.",
    "C’est une bonne journée pour étudier le traitement du langage naturel."
]


# Compute BLEU for each model
for name, preds in [
    ("MarianMT", [r[1] for r in results]),
    ("mBART-50", [r[2] for r in results]),
    ("M2M100", [r[3] for r in results])
]:
    score = bleu.compute(predictions=preds, references=[[ref] for ref in references])
    print(f"{name} BLEU Score: {score['score']:.2f}")


 BLEU SCORE COMPARISON 
MarianMT BLEU Score: 57.48
mBART-50 BLEU Score: 28.78
M2M100 BLEU Score: 46.54
